In [ ]:
#IMPORT LIBRARIES NEEDED
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras import models,layers
import tensorflow_datasets as tfds

In [ ]:
#importing dataset from tenserflow dataset and labeling it to train, test and validation
datasets, info = tfds.load(name='beans', with_info=True, as_supervised=True, split=['train','test','validation'])

In [ ]:
#info about the dataset
info

In [ ]:
# just checking some images from dataset
train, info_train = tfds.load(name='beans', with_info=True, split='test')
tfds.show_examples(info_train,train)

In [ ]:
#number of folders in dataset -> rust,angular leaf spot,healthy
len(datasets)

In [ ]:
# Before we feed our images to network, we should be resizing it to the desired size. Moreover, to improve model performance, we should normalize the image pixel value
# (keeping them in range 0 and 1 by dividing by 256).
# for mobilenet the standard input image size is 224*224*3
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return tf.image.resize(image,[224,224]), tf.one_hot(label, 3)

In [ ]:
def get_dataset(batch_size=32):
  train_dataset_scaled = datasets[0].map(scale).shuffle(1000).batch(batch_size)
  test_dataset_scaled =  datasets[1].map(scale).batch(batch_size)
  val_dataset_scaled =  datasets[2].map(scale).batch(batch_size)
  return train_dataset_scaled, test_dataset_scaled, val_dataset_scaled

In [ ]:
train_dataset, test_dataset, val_dataset = get_dataset()
train_dataset.cache()
val_dataset.cache()

In [ ]:
#building model
mobile=keras.applications.mobilenet.MobileNet(include_top=False,input_shape=(224,224,3),classifier_activation='softmax')

In [ ]:
mobile.summary()

In [ ]:
from keras.layers import Dense 

In [ ]:
# add a global spatial average pooling layer
x = mobile.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)


# and a logistic layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

In [ ]:
model=tf.keras.Model(inputs=mobile.input,outputs=predictions)
model.summary()

In [ ]:
for layer in model.layers[:-10]:  
  layer.trainable=False

In [ ]:
#model compiling using "adam"/"sgd" optimiser
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


In [ ]:
#avoid overfitting of model:
from keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=7)

In [ ]:
#training the model
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset)     
history

In [ ]:
result=model.evaluate(test_dataset)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']


plt.plot(acc,label="Training accuracy")
plt.plot(val_acc,label="Validation accuarcy")
plt.legend()
plt.show()

plt.plot(loss,label="Training loss")
plt.plot(val_loss,label="Validation loss")
plt.legend()
plt.show()

In [ ]:
for test_sample in datasets[1].take(10):  
  image, label = test_sample[0], test_sample[1]
  image_scaled, label_arr= scale(test_sample[0], test_sample[1])
  image_scaled = np.expand_dims(image_scaled, axis=0)   

  img = tf.keras.preprocessing.image.img_to_array(image)                    
  pred=model.predict(image_scaled)
  print(pred)
  plt.figure()
  plt.imshow(image)
  plt.show()
  print("Actual Label: %s" % info.features["label"].names[label.numpy()])
  print("Predicted Label: %s" % info.features["label"].names[np.argmax(pred)])

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
import pandas as pd

In [ ]:
#spot, Rust, Healthy
for f0,f1 in datasets[1].map(scale).batch(200):
  y=np.argmax(f1, axis=1)
  y_pred=np.argmax(model.predict(f0),axis=1)
  print(tf.math.confusion_matrix(labels=y, predictions=y_pred, num_classes=3))